In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential , load_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
import os
import shutil

# Define the path to the original dataset
data_path = '/content/gdrive/MyDrive/AICapstoneProjectData/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'
# Define the path to the output directory
output_path = './dataset'

# Define the ratios for train/val/test splits
train_ratio = 0.8
test_ratio = 0.2

# Create the output directories if they do not exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
for dir_name in ['train', 'test']:
    dir_path = os.path.join(output_path, dir_name)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for class_name in ['Cyst','Normal','Stone','Tumor']:
        class_path = os.path.join(dir_path, class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)

# Loop over each class and copy files to train/val/test directories
for class_name in ['Cyst', 'Normal', 'Stone', 'Tumor']:
    class_path = os.path.join(data_path, class_name)
    files = os.listdir(class_path)
    num_files = len(files)
    num_train = int(num_files * train_ratio)
    num_test = num_files - num_train
    for i, file_name in enumerate(files):
        src_path = os.path.join(class_path, file_name)
        if i < num_train:
            dst_path = os.path.join(output_path, 'train', class_name, file_name)
        else:
            dst_path = os.path.join(output_path, 'test', class_name, file_name)
        shutil.copy(src_path, dst_path)

In [12]:
import numpy as np
from sklearn.model_selection import cross_val_score,KFold
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Set up the image data generators
train_path = './dataset/train'
test_path = './dataset/test'
img_size = (224,224)
batch_size = 10
test_batch_size = 10
test_steps = 1

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    preprocessing_function=preprocess_input,
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size
)
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=test_batch_size
)

# Load the preprocessed image data and labels into numpy arrays
train_data = train_generator.next()[0]
train_labels = train_generator.next()[1]
test_data = test_generator.next()[0]
test_labels = test_generator.next()[1]

# Load the preprocessed image data and labels into numpy arrays
data = train_generator.next()[0]
labels = train_generator.next()[1]

# Reshape the numpy arrays to flatten the images
data = data.reshape(data.shape[0], -1)

# Instantiate the KNN classifier with the desired number of neighbors
k = 5
knn = KNeighborsClassifier(n_neighbors=k)

knn.fit(data,np.argmax(labels,axis=1))

# Use KFold cross-validation to evaluate the performance of the KNN classifier on the training data
kf = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, data, np.argmax(labels, axis=1), cv=kf)

# Print the mean and standard deviation of the cross-validation scores as the training accuracy
print("Training accuracy: {:.2f}% +/- {:.2f}%".format(np.mean(scores)*100, np.std(scores)*100))

# Load the preprocessed image data and labels into numpy arrays
test_data = test_generator.next()[0]
test_labels = test_generator.next()[1]

# Reshape the numpy arrays to flatten the images
test_data = test_data.reshape(test_data.shape[0], -1)

# Predict the labels of the test data using the KNN classifier
predictions = knn.predict(test_data)

# Calculate the accuracy of the KNN classifier on the test data
accuracy = np.mean(predictions == np.argmax(test_labels, axis=1))

print("Testing accuracy: {:.2f}%".format(accuracy*100))

Found 9955 images belonging to 4 classes.
Found 2492 images belonging to 4 classes.
Training accuracy: 60.00% +/- 20.00%
Testing accuracy: 100.00%


In [13]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
# Generate the confusion matrix
cm = confusion_matrix(np.argmax(test_labels,axis=1),predictions)
print("Confusion matrix:")
print(cm)



Confusion matrix:
[[10]]


In [14]:
# Generate the classification report
cr = classification_report(np.argmax(test_labels,axis=1),predictions)
print("Classification report:")
print(cr)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

